# Game sessions

## Preparation

In [ ]:
%run "../Static data/Google form correct answers.ipynb"

## Functions

In [ ]:
#RedMetrics
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'
# source: https://stackoverflow.com/questions/42047994/regex-how-to-find-a-guid-in-a-long-string

GUIDpattern = '^[0-9a-f]{8}-[0-9a-f]{4}-[1-5][0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}$'

def isGUIDFormat( guid ):
    return re.search(GUIDpattern, guid)

def getRandomRedMetricsGUID( _rmDF = rmdf152 ):
    _uniqueUsers = _rmDF['userId'].dropna().unique()
    _userCount = len(_uniqueUsers)
    _guid = ''
    while (not isGUIDFormat(_guid)):
        _userIndex = randint(0,_userCount-1)
        _guid = _uniqueUsers[_userIndex]
    return _guid

def getRandomSessionGUID( _userId = '', _rmDF = rmdf152 ):    
    rmId = _userId
    if( not(isGUIDFormat(_userId))):
        rmId = getRandomRedMetricsGUID()
    
    _uniqueSessions = getAllSessionsOfUser( _rmDF, rmId )
       
    _sessionsCount = len(_uniqueSessions)
    _guid = ""
    
    if(0 != _sessionsCount):
        _sessionIndex = randint(0,_sessionsCount-1)
        _guid = _uniqueSessions.iloc[_sessionIndex,1]
    else:
        print("no sessions for userId " + rmId)
    
    return _guid

In [ ]:
def getAllSessionsOfUser( _rmDF, userId, dropna=True ):
    _sessionsList = getAllSessions( _rmDF, dropna )
    return getAllSessionsOfUserOptimized(_sessionsList, userId)

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getAllSessionsOfUserOptimized (sessionsList, userId):
    return sessionsList[sessionsList['userId']==userId].drop_duplicates()

# includewithoutusers=True will count sessions that do not have any userId attached
def countSessions( userId, includewithoutusers, sessionsList=[], _rmDF = rmdf152):
    # List of associated sessions
    if(len(sessionsList) == 0):
        sessionsList = getAllSessionsOfUser( _rmDF, userId, True )
    if(not includewithoutusers):
        sessionsList = sessionsList.dropna(how='any')
    return sessionsList['sessionId'].nunique()

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getSessionsCountOptimized( _rmDF, userId):
    return getAllSessionsOfUserOptimized( _rmDF, userId ).shape[0]

def getUserSessionsCounts( _rmDF ):
    _allSessions = getAllSessions( _rmDF, True )
    return _allSessions.groupby('userId').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [ ]:
# get datetime of first significant event
# _userId is assumed to be in RedMetrics format
# returns a pandas._libs.tslib.Timestamp
def getFirstEventDate( _userId, _rmDF = rmdf152 ):
    _sessions = getAllSessionsOfUser(_rmDF, _userId, True)
    
    _firstGameTime = pd.Timestamp.max.tz_localize('utc')

    for session in _sessions['sessionId']:
        _timedEvents = _rmDF[_rmDF['sessionId']==session]
        
        # drops irrelevant events like 'start' and 'configure' ones
        _timedEvents = _timedEvents.dropna(subset=['section'])
        
        if(len(_timedEvents) > 0):
            _timedEvents['userTime'] = _timedEvents['userTime'].map(lambda t: pd.to_datetime(t, utc=True))

            _earliest = _timedEvents['userTime'].min()
            if(_earliest < _firstGameTime):
                _firstGameTime = _earliest

    return _firstGameTime